# 🗄️Text to SQL Agent

In this notebook, we'll make an AI agent to interact with your SQL database using LLMs, google's gemini in particular

Your takeaway:
1. Learn to interact with SQL using python.
2. Create an AI Agent __without__ langchain, but using gemini directly.
3. Learn how to let your AI agent interact with SQL, so you can use your database in normal english language.

## ⚙️ Import Dependencies

We will not be using langchain in this mini-project. We'll use google library directly and also import sqlite3 for our database setup.

In [3]:
import sqlite3
from google import genai

In [4]:
from google.colab import userdata
API_KEY=userdata.get('GEMINI_API_KEY')

### Configure the gemini api using the appropriate API Key.

In [5]:
client = genai.Client(api_key=API_KEY)

### To generate a SQL query from gemini

In [6]:
def get_gemini_response(question, prompt):
    response = client.models.generate_content(
        model="gemini-2.5-flash",  # Latest fast model
        contents=[prompt[0], question]
    )
    return response.text.strip()

### To execute the query given by gemini and return the response

In [7]:
def read_sql_query(sql, db):
    try:
        conn = sqlite3.connect(db)
        cur = conn.cursor()
        cur.execute(sql)
        rows = cur.fetchall()
        conn.close()
        return rows
    except sqlite3.Error as e:
        return [f"SQL Error: {e}"]

### Add dummy data to Student table in student.db

In [13]:
def create_student_db():
    conn = sqlite3.connect('student.db')
    cursor = conn.cursor()

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS STUDENT (
        NAME TEXT NOT NULL,
        CLASS TEXT NOT NULL,
        SECTION TEXT NOT NULL
    )
    ''')

    students = [
        ('Alice', 'Data Science', 'A'),
        ('Bob', 'Computer Science', 'B'),
        ('Charlie', 'Data Science', 'B'),
        ('David', 'Mathematics', 'A')
    ]

    cursor.executemany('INSERT INTO STUDENT (NAME, CLASS, SECTION) VALUES (?, ?, ?)', students)

    conn.commit()
    conn.close()
    print("student.db created with STUDENT table and example data.")

### Prompt to generate the query (few-shot prompting).

In [9]:
prompt = [
    """
    You are an expert in converting English questions to SQL query!
    The SQL database has the name STUDENT and has the following columns:
    NAME, CLASS, SECTION.

    Example 1 - How many entries of records are present?
    SQL: SELECT COUNT(*) FROM STUDENT;

    Example 2 - Tell me all the students studying in Data Science class?
    SQL: SELECT * FROM STUDENT WHERE CLASS="Data Science";

    Rules:
    - Only output valid SQL
    - Do NOT include ``` or 'sql' markers
    - SQL should be ready to run directly
    """
]

In [14]:
create_student_db()

student.db created with STUDENT table and example data.


### Give your query in natural language.

In [21]:
question = input("Input your question: ")
sql_query = get_gemini_response(question, prompt)

Input your question: How many students in section A?


In [22]:
print("\nGenerated SQL Query:")
print(sql_query)


Generated SQL Query:
SELECT COUNT(*) FROM STUDENT WHERE SECTION = "A";


### Returns result from database

In [23]:
results = read_sql_query(sql_query, "student.db")

print("\nThe Response is:")
for row in results:
    print(row)


The Response is:
(2,)
